In [10]:
import sqlalchemy

import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# Connect to PostgreSQL - Staging
DATABASE_URL_STAGING = os.getenv("DATABASE_URL_STAGING")
engine = sqlalchemy.create_engine(DATABASE_URL_STAGING,  client_encoding='utf8')
connection = engine.connect()

In [12]:
# Check Database
asd = sqlalchemy.inspect(engine)
print(asd.get_table_names())

['combined_sales_new', 'combined_sales']


In [13]:
import pandas as pd

# Select Data
query = "SELECT * FROM combined_sales_new;"
result_df = pd.read_sql(query, con=connection)
result_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,FROM
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"128 Elm St, Quebec City",Canada
1,None,None,None,None,None,"113 Oak St, Ottawa",Canada
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"116 Elm St, Winnipeg",Canada
3,176560,Google Phone,1,600,04/12/19 14:38,"149 Broadway, Edmonton",Canada
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"121 Maple St, Vancouver",Canada
...,...,...,...,...,...,...,...
30389,248146,Bose SoundSport Headphones,1,99.99,08/29/19 22:19,"868 Hickory St, San Francisco, CA 94016",USA
30390,248147,AAA Batteries (4-pack),3,2.99,08/31/19 16:26,"206 Lakeview St, Boston, MA 02215",USA
30391,248148,AA Batteries (4-pack),1,3.84,08/02/19 07:25,"568 13th St, Seattle, WA 98101",USA
30392,248149,USB-C Charging Cable,1,11.95,08/08/19 12:10,"495 Walnut St, San Francisco, CA 94016",USA


In [14]:
result_df.isnull().sum()

Order ID            87
Product             87
Quantity Ordered    87
Price Each          87
Order Date          87
Purchase Address    28
FROM                 0
dtype: int64

In [15]:
result_df = result_df.dropna()
result_df.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
FROM                0
dtype: int64

In [16]:
result_df.duplicated().sum()

np.int64(43)

In [17]:
result_df = result_df.drop_duplicates()
result_df.duplicated().sum()

np.int64(0)

In [18]:
result_df = result_df[result_df["Order ID"].str.isnumeric()]
result_df.shape

(30228, 7)

In [19]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30228 entries, 0 to 30393
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order ID          30228 non-null  object
 1   Product           30228 non-null  object
 2   Quantity Ordered  30228 non-null  object
 3   Price Each        30228 non-null  object
 4   Order Date        30228 non-null  object
 5   Purchase Address  30228 non-null  object
 6   FROM              30228 non-null  object
dtypes: object(7)
memory usage: 1.8+ MB


In [20]:
# Remove Whitespaces
result_df["Order ID"] = result_df["Order ID"].str.strip()
result_df["Product"] = result_df["Product"].str.strip()
result_df["Quantity Ordered"] = result_df["Quantity Ordered"].str.strip()
result_df["Price Each"] = result_df["Price Each"].str.strip()
result_df["Order Date"] = result_df["Order Date"].str.strip()
result_df["Purchase Address"] = result_df["Purchase Address"].str.strip()

# Fix Data Types
result_df["Order ID"] = result_df["Order ID"].astype(int)
result_df["Quantity Ordered"] = result_df["Quantity Ordered"].astype(int)
result_df["Price Each"] = result_df["Price Each"].astype(float)
result_df["Order Date"] = pd.to_datetime(result_df["Order Date"])

C:\Users\ianju\AppData\Local\Temp\ipykernel_7964\3462584182.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result_df["Order Date"] = pd.to_datetime(result_df["Order Date"])


In [21]:
usa_result_df = result_df[result_df["FROM"].str.contains("USA")]
canada_result_df = result_df[result_df["FROM"].str.contains("Canada")]

In [22]:
canada_result_df["Price Each"] = canada_result_df["Price Each"] * 0.72

C:\Users\ianju\AppData\Local\Temp\ipykernel_7964\950003836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canada_result_df["Price Each"] = canada_result_df["Price Each"] * 0.72


In [23]:
final_df = pd.concat([usa_result_df, canada_result_df], ignore_index=True)
final_df = final_df.rename(columns={"FROM": "Country"})
final_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Country
0,240221,iPhone,1,700.0000,2019-08-09 20:03:00,"714 Lake St, Portland, OR 97035",USA
1,242936,iPhone,1,700.0000,2019-08-14 09:27:00,"984 14th St, Austin, TX 73301",USA
2,244595,iPhone,1,700.0000,2019-08-03 18:09:00,"711 Center St, Seattle, WA 98101",USA
3,236670,Wired Headphones,2,11.9900,2019-08-31 22:21:00,"359 Spruce St, Seattle, WA 98101",USA
4,236671,Bose SoundSport Headphones,1,99.9900,2019-08-15 15:11:00,"492 Ridge St, Dallas, TX 75001",USA
...,...,...,...,...,...,...,...
30223,194090,Google Phone,1,432.0000,2019-04-08 17:11:00,"124 Market St, Quebec City",Canada
30224,194091,AA Batteries (4-pack),1,2.7648,2019-04-15 16:02:00,"130 Cedar Ave, Ottawa",Canada
30225,194092,AAA Batteries (4-pack),2,2.1528,2019-04-28 14:36:00,"137 Maple St, Vancouver",Canada
30226,194093,AA Batteries (4-pack),1,2.7648,2019-04-14 15:09:00,"133 Elm St, Vancouver",Canada


In [24]:
# Close the connection
connection.close()

In [25]:
# Connect to PostgreSQL - Final
DATABASE_URL_FINAL = os.getenv("DATABASE_URL_FINAL") 
engine = sqlalchemy.create_engine(DATABASE_URL_FINAL,  client_encoding='utf8')
connection = engine.connect()

In [26]:
# Save DataFrame to Database
final_df.to_sql("sales_data_duckdb", con=connection, if_exists="replace", index=False)

228

In [27]:
# Verify the data was saved correctly
query = "SELECT * FROM sales_data_duckdb LIMIT 5;"
check_df = pd.read_sql(query, con=connection)
check_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Country
0,240221,iPhone,1,700.00,2019-08-09 20:03:00,"714 Lake St, Portland, OR 97035",USA
1,242936,iPhone,1,700.00,2019-08-14 09:27:00,"984 14th St, Austin, TX 73301",USA
2,244595,iPhone,1,700.00,2019-08-03 18:09:00,"711 Center St, Seattle, WA 98101",USA
3,236670,Wired Headphones,2,11.99,2019-08-31 22:21:00,"359 Spruce St, Seattle, WA 98101",USA
4,236671,Bose SoundSport Headphones,1,99.99,2019-08-15 15:11:00,"492 Ridge St, Dallas, TX 75001",USA


In [8]:
# Verify the data was saved correctly
query_ext = "SELECT * FROM pg_extension;"
pd.read_sql(query_ext, con=connection)

,oid,extname,extowner,extnamespace,extrelocatable,extversion,extconfig,extcondition
0,13564,plpgsql,10,11,False,1.0,None,None
1,16398,pg_duckdb,16385,2200,False,0.2.0,None,None


In [28]:
connection.close()